# Mitiq Tutorial

## Executors

In [2]:
import cirq


def execute(circuit, noise_level=0.005):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    with depolarizing noise."""
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    return (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix[0, 0]
        .real
    )

## ZNE

In [8]:
from mitiq import zne
from mitiq.benchmarks import generate_rb_circuits

circuit = generate_rb_circuits(n_qubits=2, num_cliffords=20)[0]

true_value = execute(circuit, noise_level=0.0)
noisy_value = execute(circuit)
zne_value = zne.execute_with_zne(circuit, execute)

print(f"Error w/o  Mitiq: {abs((true_value - noisy_value) / true_value):.3f}")
print(f"Error w Mitiq:    {abs((true_value - zne_value) / true_value):.3f}")

Error w/o  Mitiq: 0.558
Error w Mitiq:    0.313


## Calibration

In [9]:
import numpy as np
from mitiq import MeasurementResult


def execute_with_depolarizing(circuit, noise_level=0.001):
    circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    result = cirq.DensityMatrixSimulator().run(circuit, repetitions=100)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)

In [10]:
from mitiq import Calibrator

cal = Calibrator(execute_with_depolarizing, frontend="cirq")

cal.run(log=True)

| performance | circuit | method | extrapolation | scale factors | scale_method         |
| ----------- | ------- | ------ | ------------- | ------------- | -------------------- |
|     ❌      |   ghz   |  ZNE   | Richardson    | 1.0, 2.0, 3.0 | fold_global          |
|     ✅      |   ghz   |  ZNE   | Richardson    | 1.0, 3.0, 5.0 | fold_global          |
|     ❌      |   ghz   |  ZNE   | Linear        | 1.0, 2.0, 3.0 | fold_global          |
|     ❌      |   ghz   |  ZNE   | Linear        | 1.0, 3.0, 5.0 | fold_global          |
|     ❌      |   ghz   |  ZNE   | Richardson    | 1.0, 2.0, 3.0 | fold_gates_at_random |
|     ❌      |   ghz   |  ZNE   | Richardson    | 1.0, 3.0, 5.0 | fold_gates_at_random |
|     ✅      |   ghz   |  ZNE   | Linear        | 1.0, 2.0, 3.0 | fold_gates_at_random |
|     ❌      |   ghz   |  ZNE   | Linear        | 1.0, 3.0, 5.0 | fold_gates_at_random |
|     ❌      |    w    |  ZNE   | Richardson    | 1.0, 2.0, 3.0 | fold_global          |
|     ✅      |    w

In [11]:
cal.best_strategy()

{'technique': 'ZNE', 'factory': 'RichardsonFactory', 'scale_factors': [1.0, 3.0, 5.0], 'scale_method': 'fold_global'}